In [1]:
import os, gc
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
import torch

/home/landon/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
save_dir = "models/llama-2-7b-chat-hf-qint8-4th"
quantized_model_path = os.path.join(save_dir, "quantized_model.pth")

In [3]:
gc.collect()

20

In [4]:
# Directory where the quantized model and tokenizer are saved
save_dir = "models/llama-2-7b-chat-hf-qint8-4th"
quantized_model_path = os.path.join(save_dir, "quantized_model.pth")

# Load the configuration
config = AutoConfig.from_pretrained(save_dir)

# Initialize the original model instance with the loaded configuration
model_for_loading = AutoModelForCausalLM.from_config(config)

# Apply dynamic quantization to match the saved quantized model's structure
quantized_model_for_loading = torch.quantization.quantize_dynamic(
    model_for_loading, {torch.nn.Linear}, dtype=torch.qint8
)

In [5]:
gc.collect()

5625

In [6]:
# Load the quantized model's state dictionary
quantized_model_state_dict = torch.load(quantized_model_path, map_location=torch.device('cpu'))
quantized_model_for_loading.load_state_dict(quantized_model_state_dict)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(save_dir)

# The quantized model is now ready for use


/home/landon/.local/lib/python3.10/site-packages/torch/_utils.py:383: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  device=storage.device,


In [32]:
prompt = "You are a helpful chatbot that speaks predominantly in English. Please resond in a friendly manner and help ansering any questions I may have. Qustions and conversations may will be included after this introduction. Here is my question:"

# Example text to encode
in_text = "Hello, who was the president in 2010?"

text = f"{prompt} \n \n {in_text}"

# Encode the text, including the attention mask
encoded_input = tokenizer(text, return_tensors="pt", truncation=True)
input_ids = encoded_input["input_ids"]
attention_mask = encoded_input["attention_mask"]

# Ensure the pad_token_id is set for the model if it's not already
if quantized_model_for_loading.config.pad_token_id is None:
    quantized_model_for_loading.config.pad_token_id = quantized_model_for_loading.config.eos_token_id

# Run inference, providing both input_ids and attention_mask
with torch.no_grad():
    outputs = quantized_model_for_loading.generate(
    input_ids,
    attention_mask=attention_mask,
    pad_token_id=quantized_model_for_loading.config.pad_token_id,
    max_length=100,  # Adjust max_length for longer responses
    temperature=0.7,  # Adjust temperature for creativity
    top_p=0.9,  # Adjust nucleus sampling's cumulative probability cutoff
    num_beams=5,  # Use beam search with specified number of beams
    do_sample=True  # Use sampling
)


# Decode the generated text
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(generated_text)


You are a helpful chatbot that speaks predominantly in English. Please resond in a friendly manner and help ansering any questions I may have. Qustions and conversations may will be included after this introduction. Here is my question: 
 
 Hello, who was the president in 2010?

Hello! I'm here to help you with any questions you may have. To answer your question, the President of the United States in
